In [2]:
# !pip install fosforml

In [2]:
# pip show fosforml

## Sklearn Models Registration

### session creation

In [21]:
from fosforml.model_manager.snowflakesession import get_session, get_connection_params
session = get_session()
connection_params = get_connection_params()

### modeling

#### Multiclass classification

In [7]:
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

m_dataset = load_iris() 
feature_names = m_dataset.feature_names
u_features = [col.replace(" ","_").replace("(","").replace(")","") for col in feature_names]
m_df = pd.DataFrame(m_dataset.data,columns=u_features)
m_df["Target"] = m_dataset.target


x_train,x_test,y_train,y_test = train_test_split(m_df.iloc[:,:-1],m_df['Target'])
model = RandomForestClassifier()
model.fit(x_train,y_train)

y_pred = pd.DataFrame(model.predict(x_test),columns=["Predicted"])

## Custom Function

In [41]:
def score_func(model,input_dataframe:pd.DataFrame) -> pd.DataFrame:
    return input_dataframe

In [11]:
from snowflake.ml.model import custom_model
from snowflake.ml.model.custom_model import ModelContext

mc = custom_model.ModelContext(
    artifacts={
        'input_cols': [],
        'output_cols' : [],
        'label_cols' : [],
        'output_prob_cols' : []
    },
    models={
        'm1': model,
        'feature_preproc': score_func
    }
)

In [17]:
class ExamplePipelineModel(custom_model.CustomModel):
    def __init__(self, context: ModelContext) -> None:
        super().__init__(context)
        
    @custom_model.inference_api
    def predict(self, input_data: pd.DataFrame) -> pd.DataFrame:
        model_output = self.context.model_ref('m1').predict(input_data)
        return pd.DataFrame({
            'output': model_output })

In [19]:
my_model_pipeline = ExamplePipelineModel(mc)
my_model_pipeline.predict(x_test).head()

,output
0,2
1,0
2,0
3,0
4,0


In [36]:
# !env | grep connection_details

In [26]:
from snowflake.ml.registry import Registry
reg = Registry(session=session, database_name="INSIGHT_DESIGNER_SPCS", schema_name="INSIGHT_SPCS_SCHEMA")
mv = reg.log_model(my_model_pipeline,
            model_name="my_custom_model_pipeline",
            version_name="v1",
            conda_dependencies=["scikit-learn"],
            comment="My Custom ML Model Pipeline",
            sample_input_data=x_train)

In [35]:
# reg.show_models()

In [27]:
output_df = mv.run(session.create_dataframe(x_train))

In [29]:
output_df.to_pandas()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,output
0,4.7,3.2,1.6,0.2,0
1,5.5,2.4,3.7,1.0,1
2,5.1,3.7,1.5,0.4,0
3,5.2,3.5,1.5,0.2,0
4,7.2,3.6,6.1,2.5,2
...,...,...,...,...,...
107,5.5,2.5,4.0,1.3,1
108,5.0,3.4,1.5,0.2,0
109,6.4,3.1,5.5,1.8,2
110,5.6,2.8,4.9,2.0,2


In [ ]:
from snowflake.ml.model import custom_model
from snowflake.ml.model.custom_model import ModelContext

mc = custom_model.ModelContext(
    models={
        'm1': model
    }
)

class CustomModel(custom_model.CustomModel):
    def __init__(self, context: ModelContext) -> None:
        super().__init__(context)
        
    @custom_model.inference_api
    def predict(self, input_data: pd.DataFrame) -> pd.DataFrame:
        model_output = self.context.model_ref('m1').predict(input_data)
        return pd.DataFrame({
            'output': model_output })
        

In [40]:
# my_model_pipeline.

## Model registration

In [37]:
from fosforml import register_model
register_model(
  model_obj=my_model_pipeline,
  session=session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  dataset_name="RandomMulticlassMOdelForTesting",
  dataset_source="Snowflake",
  name="RandomMultieefefefclassMOdelForTesting16July",
  description="This is a test sklearn model",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Exception: Failed to get model performance metrics. 'feature_names'